In [56]:
import time as time
import datetime as datetime
import pandas as pd
import yfinance as yf
import matplotlib as plt
import talib
from backtesting import Strategy, Backtest
from backtesting.lib import crossover

In [57]:
#Ticker in url
ticker = 'TSLA'

#Timeperiods of data set "Y/M/D/time"
#this will help when selecting the desired dates and will pull the data set from yahoo finance.
period1 = int(time.mktime(datetime.datetime(2018, 12, 1, 23, 59).timetuple()))
period2 = int(time.mktime(datetime.datetime(2022, 12, 31, 23, 59).timetuple()))
interval = '1d' # 1wk, 1m

#Yahoo Finance url
url = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'


In [58]:
#shaping data frame to backtesting requirements
df = pd.read_csv(url)
columns = ['Date', 'Open', 'High', 'Low', 'Close', 'adj close', 'Volume']
df.columns = columns

In [59]:
#index data frame to date time index to fit backtesting.py
#df requirements
df = df.set_index(pd.DatetimeIndex(df['Date'].values))

In [60]:
#dropping columns that are not necessary
df.drop('Date', inplace=True, axis=1)
df.drop('adj close', inplace=True, axis=1)
df

,Open,High,Low,Close,Volume
2018-12-03,24.000000,24.400000,23.466667,23.899332,124597500
2018-12-04,23.736668,24.578667,23.466667,23.980000,126928500
2018-12-06,23.733999,24.492001,23.384001,24.204000,117637500
2018-12-07,24.600000,25.299334,23.843332,23.864668,172668000
2018-12-10,24.000000,24.398666,23.541332,24.343332,99202500
...,...,...,...,...,...
2022-12-23,126.370003,128.619995,121.019997,123.150002,166989700
2022-12-27,117.500000,119.669998,108.760002,109.099998,208643400
2022-12-28,110.349998,116.269997,108.239998,112.709999,221070500
2022-12-29,120.389999,123.570000,117.500000,121.820000,221923300


In [61]:
#the overall trading strategy function
class RSI(Strategy): 
    #define upper and lower bands of indicator. If the stock price
    #dips below 30 it buys and above 70 it sells
    upper_bound = 70
    lower_bound = 30
    
    #defines the premade trading parameters imported from talib
    #(trading parameter, data column being used, trading window)
    def init(self):
        self.rsi = self.I(talib.RSI, self.data.Close, 14)
    
    def next(self):
        
        if crossover(self.rsi, self.upper_bound):
            #if this statement is true the below command signals a sell.
            self.sell()
        
        elif crossover(self.lower_bound, self.rsi):
            #buy command 
            self.buy()
                       

#bt variable runs the backtest dependant on the data, strategy, and cash
#other parameters can be added to more complex strategies. Refer to 
#backtesting.py on github
bt = Backtest(df, RSI, cash = 3_000)
stats = bt.run()

#the plotting function does not work in python 3.8.7 so it needs to be 
#run in a earlier python like python 3.6 to graph the trades 
bt.plot()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p12747', ...)